In [37]:
print("OK")

OK


# Q&A over the Code Base to Understand How it Works

In [38]:
from git import Repo
import os

from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [39]:
%pwd

'/home/fahad/Documents/Projects/SPL Excessories/SPL3/Code Analyzer LLM/research'

In [40]:
# !mkdir test_repo

In [41]:
repo_path = "test/python-type-hinting-main"
# repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", to_path=repo_path)

In [42]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [43]:
documents = loader.load()

In [44]:
documents

[Document(metadata={'source': 'test/python-type-hinting-main/test.py', 'language': <Language.PYTHON: 'python'>}, page_content='from dataclasses import dataclass\nfrom typing import TypedDict\n\n\n# Define a TypedDict to explicitly type the car_data dictionary\nclass CarData(TypedDict):\n    model: str\n    year: int\n    trim_package: str\n    mass: float\n    engine_type: str\n    engine_size: str\n    engine_location: str\n    cylinders: int\n    turbo: bool\n    unit_price: float\n    colour: str\n    wheels: int\n    seats: int\n\n\nclass EngineAttributes(TypedDict):\n    engine_type: str\n    engine_size: str\n    cylinders: float\n    turbo: bool\n    engine_location: str\n\n\n@dataclass\nclass Car:\n    """Class for a particular car item in inventory."""\n\n    model: str\n    year: int\n    trim_package: str\n    mass: float\n    engine_type: str\n    engine_size: str\n    engine_location: str\n    cylinders: int\n    turbo: bool\n    unit_price: float\n    colour: str\n    whe

In [45]:
len(documents[0].page_content)

1777

In [46]:
documents[0]

Document(metadata={'source': 'test/python-type-hinting-main/test.py', 'language': <Language.PYTHON: 'python'>}, page_content='from dataclasses import dataclass\nfrom typing import TypedDict\n\n\n# Define a TypedDict to explicitly type the car_data dictionary\nclass CarData(TypedDict):\n    model: str\n    year: int\n    trim_package: str\n    mass: float\n    engine_type: str\n    engine_size: str\n    engine_location: str\n    cylinders: int\n    turbo: bool\n    unit_price: float\n    colour: str\n    wheels: int\n    seats: int\n\n\nclass EngineAttributes(TypedDict):\n    engine_type: str\n    engine_size: str\n    cylinders: float\n    turbo: bool\n    engine_location: str\n\n\n@dataclass\nclass Car:\n    """Class for a particular car item in inventory."""\n\n    model: str\n    year: int\n    trim_package: str\n    mass: float\n    engine_type: str\n    engine_size: str\n    engine_location: str\n    cylinders: int\n    turbo: bool\n    unit_price: float\n    colour: str\n    whee

In [47]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [48]:
texts = documents_splitter.split_documents(documents)

In [49]:
len(texts[0].page_content)

392

In [50]:
len(texts)

33

In [51]:
from dotenv import load_dotenv
load_dotenv() 

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [52]:
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY

In [53]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings=OpenAIEmbeddings(disallowed_special=())
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [54]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')

In [55]:
# vectordb.persist()

In [88]:
from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatOpenAI(model_name="gpt-4")
# llm = ChatOpenAI()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.5, max_tokens=4018, google_api_key=GOOGLE_API_KEY)

In [89]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [90]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [ ]:
question = ''' 
The following object contains type related issue.
{rule_id: 'Undefined or invalid type [11]', message: 'Annotation `dict` is not defined as a type.', warning_line: 'def obtain_price_list(self, price_list: dict[str, float]):', source_code: 'No function definition found'}

You should reply as the following format only. Don't engage in conversation and add any extra explanation.
1. Cause
2. Solution in Code
3. Explain solution in bullet point
'''

In [92]:
result = qa(question)
print(result['answer'])

1. **Cause:** The `dict` type annotation needs to be imported from the `typing` module when used in type hints before Python 3.9.  In Python 3.9+, `dict` can be used directly as a type hint.

2. **Solution in Code:**
```python
from typing import dict

def obtain_price_list(self, price_list: dict[str, float]):
    """Obtain price list from head office."""
    self.price_list = price_list

```

3. **Explanation:**

* The `from typing import dict` line imports the `dict` type hint. This explicitly tells Python that `price_list` should be treated as a dictionary.
* This solution works for Python versions prior to 3.9.  If using 3.9 or later, the import is unnecessary.



In [93]:
question = "As newbie from other programming background don't have a clear understing of python types. So explain the previous types with example from different coding language so that they can be familiar with python type hints"
result = qa(question)
print(result['answer'])

Python's type hinting system is a way to add static typing information to your Python code, even though Python remains a dynamically typed language at its core.  Type hints are primarily used for static analysis tools (like MyPy), IDEs (for better code completion and error detection), and improved code readability.  They don't generally change the runtime behavior of your program.

Let's focus on `dict` annotations and draw parallels to other languages:

**Python Type Hints for `dict`**

```python
# Python 3.9+ (using built-in types)
my_dict: dict[str, int] = {"apple": 1, "banana": 2}

# Python 3.6 - 3.8 (using typing module)
from typing import Dict
my_dict: Dict[str, int] = {"apple": 1, "banana": 2}  
```

This signifies that `my_dict` is expected to be a dictionary where keys are strings and values are integers.

**Comparison to other languages:**

* **Java:**  Python's `dict[str, int]` is similar to Java's `Map<String, Integer>`.  Both specify a map (or dictionary) where keys are st

In [95]:
question = '''
This is an object containing pyre error message and a warning line
{rule_id: 'Undefined or invalid type [11]', message: 'Annotation `dict` is not defined as a type.', warning_line: 'def obtain_price_list(self, price_list: dict[str, float]):', source_code: Null}
To get the full solution we need the source code as well.

Extract the complete source code and print it
'''
result = qa(question)
print(result['answer'])

I can infer parts of the code, but I don't have enough information to reconstruct the complete source code.  I'm missing the definitions of `Car`, `Showroom`, and `open_showrooms`, as well as the context of where `calculate_stock_value` is defined. I also don't know the contents of the `inventory` variable used within `calculate_stock_value`.

Here's a best guess, with placeholders for the missing parts:

```python
import math
import typing
from dataclasses import dataclass

# Placeholder for Car class definition
class Car:  # Missing complete definition
    def __init__(self, *args):
        # ... handling of arguments
        pass
    def get_engine_attributes(self):
        # ... implementation
        pass

# Placeholder for Showroom class definition
class Showroom:
    def __init__(self):
        self.price_list = {}
        self.inventory = {} # This is a guess, it's not defined in provided snippets
        self.total_stock_value = 0

    def obtain_price_list(self, price_list: d